In [1]:
!pip install transformers[sentencepiece]

In [2]:
pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to c:\users\razer\appdata\local\temp\pip-req-build-g84uil3n
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer 'C:\Users\RAZER\AppData\Local\Temp\pip-req-build-g84uil3n'


In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer


# calling BanglaT5 model
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5", use_fast=False)

In [11]:
"""# Mounting colab with Google Drive

from google.colab import drive
drive.mount('/content/drive')"""

"# Mounting colab with Google Drive\n\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [12]:
# It it recommended to not change the dataset's files' name

# 'base' variable has to store the path to the folder where datasets are uploaded/available
# so set the 'base' variable path to the folder of the where you have uploaded the datasets
base = 'C:/Users/RAZER/Documents/Sartaj/BQA'        # sample

# 'checkpoint_dir' variable has to store the path to the folder where checkpoints will be saves
# IT IS RECOMMENDED TO SET checkpoint_dir SAME AS base
# so set the 'checkpoint_dir' variable path to the folder of the where you want the check points to be saved
checkpoint_dir = 'C:/Users/RAZER/Documents/Sartaj/BQA'          #sample
#Documents/Sartaj/models
# 'train_filename' variable has to store the filename of the Train set
# so set the 'train_filename' variable as the name of the Train set's filename
train_filename = 'Train.json'        #sample

# 'validation_file_name' variable has to store the filename of the Validation set
# so set the 'validation_file_name' variable as the name of the validation set's filename
validation_file_name = 'Validation.json'     #sample

In [13]:
# Loading the Train.json file for train set of BanglaRQA

import json
import os

f = open(os.path.join(base,train_filename))
  
data_test = json.load(f)

data_test.keys()
  

dict_keys(['data'])

In [14]:
data = data_test['data']
len(data)

2400

In [15]:
data[0]

{'passage_id': 'bn_wiki_2977',
 'context': 'ফাজিল পরীক্ষা বাংলাদেশ ও ভারতের আলিয়া মাদ্রাসায় অনুষ্ঠিত একটি সরকারি পরীক্ষা। ফাজিল পরীক্ষা বাংলাদেশে ডিগ্রি সমমানের, কখনো স্নাতক সমমানের একটি পরীক্ষা, যা একটি ফাজিল মাদ্রাসায় অনুষ্ঠিত হয়ে থাকে। তবে ভারতে ফাজিল পরীক্ষাকে উচ্চ মাধ্যমিক শ্রেণীর (১১ বা ১২ ক্লাস) মান বলে বিবেচিত করা হয়। ফাজিল পরীক্ষা বাংলাদেশ ভারত ও পাকিস্তানের সরকারি স্বীকৃত আলিয়া মাদরাসায় প্রচলিত রয়েছে। বাংলাদেশের ফাজিল পরীক্ষা ইসলামি আরবি বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়ে থাকে ও ভারতের ফাজিল পরীক্ষা পশ্চিমবঙ্গ মাদ্রাসা শিক্ষা পর্ষদের অধীনে অনুষ্ঠিত হয়ে থাকে।\n\n১৯৪৭ সালে ঢাকা আলিয়া মাদ্রাসা ঢাকায় স্থানান্তরের পূর্বে বাংলাদেশ ও ভারতের ফাজিল পরীক্ষা কলকাতা আলিয়া মাদ্রাসার অধীনে অনুষ্ঠিত হতো। ফাযিল পরীক্ষা বর্তমানে ইসলামি আরবী বিশ্ববিদ্যালয়ের অধীনে অনুষ্ঠিত হয়। যা পূর্বে মাদরাসা বোর্ড ও ইসলামি বিশ্ববিদ্যালয়ের আধীনে অনুষ্ঠিত হত। মাদ্রাসা-ই-আলিয়া ঢাকায় স্থানান্তরিত হলে ১৯৪৮ সালে মাদ্রাসা বোর্ডের ফাজিলগুলো পরীক্ষা ঢাকা বিশ্ববিদ্যালয় কর্তৃক গৃহীত হতো। ১৯৭৫ সালের 

In [16]:
import pandas

context = []
question = []
answer = []


path = "C:/Users/RAZER/Documents/Sartaj/new_data/New_data.csv"

df = pandas.read_csv(path)
df = df.dropna()
context = list(df['Context'])
question = list(df['Question'])
answer = list(df['Answer'])




for i in range(len(data)):
    for j in range(len(data[i]['qas'])):
        context.append(normalize(data[i]['context']))
        question.append(normalize(data[i]['qas'][j]['question_text']))
        answer.append(normalize(data[i]['qas'][j]['answers']['answer_text'][0]))

In [18]:
print(len(context))
print(len(question))
print(len(answer))

23951
23951
23951


In [19]:
# encoding the inputs and outputs

train_encodings = tokenizer(question, context, max_length=1024, padding="max_length", truncation="only_second", return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
train_answer_encodings = tokenizer(answer, max_length=256, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

In [20]:
train_answer_encodings = train_answer_encodings['input_ids']
train_answer_encodings[train_answer_encodings==0] = -100

In [21]:
train_encodings.update({'train_answer_encodings': train_answer_encodings})

In [22]:
train_encodings

{'input_ids': tensor([[ 1456,  1681,  4186,  ...,     0,     0,     0],
        [ 1998,  3442, 24240,  ...,     0,     0,     0],
        [   96,    14, 18309,  ...,     0,     0,     0],
        ...,
        [  265,     8,   398,  ...,     0,     0,     0],
        [ 1119,   845,  1103,  ...,     0,     0,     0],
        [ 1119,   845,  1103,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'train_answer_encodings': tensor([[ 1018,  1542,     1,  ...,  -100,  -100,  -100],
        [ 5549,    57, 15357,  ...,  -100,  -100,  -100],
        [27073,    58,  5851,  ...,  -100,  -100,  -100],
        ...,
        [  503, 12897,   584,  ...,  -100,  -100,  -100],
        [    1,  -100,  -100,  ...,  -100,  -100,  -100],
        [    1,  -100,  -100,  ...,  -100,  -100,  -100]]

In [23]:
# Loading the validation set of BanglaRQA

import json
import os

f_val = open(os.path.join(base,validation_file_name))
  
data_val = json.load(f_val)

data_val.keys()
  

dict_keys(['data'])

In [24]:
data_val = data_val['data']

In [25]:
context_val = []
question_val = []
answer_val = []

for i in range(len(data_val)):
    for j in range(len(data_val[i]['qas'])):
        context_val.append(normalize(data_val[i]['context']))
        question_val.append(normalize(data_val[i]['qas'][j]['question_text']))
        answer_val.append(normalize(data_val[i]['qas'][j]['answers']['answer_text'][0]))

In [27]:
context_val[0]

'আস্যাইরিয়ান ও ব্যাবলিয়ান সাহিত্যে সরগন তার অবনমিত অবস্থান থেকে ক্ষমতায় উত্থান ও মেসোপটেমিয়া অভিযানের জন্য কিংবদন্তি গল্পের মুল উপজীব্যে পরিনত হয়েছিলেন, এরকম আরও কিছু কিছু আংশিক কিংবদন্তী উপাখ্যান ছাড়াও সারগনের খোদ নিজের অনেক লিপি আছে যদিও তার বেশিরভাগ পরবর্তী সংস্করনগুলি থেকে নেওয়া। ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফলক এর অংশবিশেষ আছে যা সুসা (যেখানে এগুলি খুব সম্ভবত মেসোপটেমিয়া ত্থেকে দ্বাদশ শতাব্দীতে স্থানান্তরিত করা হয়েছিল) থেকে পুনঃউদ্ধার করা হয়েছিল। দৃশ্যত সরগন সেমেটিক (আক্কাদীয়ান) ভাষার লিপির লিখিত আকারে প্রসার ঘটিয়েছিলেন, তিনি আক্কাদ শহর প্রতিষ্ঠিত করার প্রথমদিকে নিজেকে প্রায়শয়ই আক্কাদীয়ান রাজা হিসিবে প্রচার করতেন, পরে তিনি কোন এক সময় কিস শহর অধিগ্রহণ করে নেন, পরবর্তীতে মেসোপটেমিয়ার বৃহদাংশও দখল করে নেন, এবং ক্রমে ক্রমে "সরগন, আক্কদীয়ান রাজা, ইনান্নার তত্ত্বাবধায়ক, কিস এর রাজা, আনুর স্থলাভিষিক্ত, রাজ্যের[মেসোপটেমিয়া] রাজা, এনলিলের রাজ্যপাল[এনসি]" নামে নিজেকে প্রচার করে ছিলেন। যদিও সুমেরিয়ান রাজাদের তালিকার অনেক অনুলিপিতে সারগনের শাসনকাল ৫৬, ৫৫ বা ৫৪ বছর বল

In [28]:
from collections import Counter

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    common_tokens = sum(common_tokens.values())
    
    # if there are no common tokens then f1 = 0
    if common_tokens == 0:
        return 0
    
    prec = 1.0 * common_tokens / len(pred_tokens)
    rec = 1.0 * common_tokens / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)


In [29]:
import numpy

def get_answer(question,context):
  source_encoding=tokenizer(
    question,
    context,
    max_length=1024,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt").to(device)
  
  #print(source_encoding)
  
  generated_ids=model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams = 1,
      max_length=256,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping= True,
      use_cache = True,
  )
  
  #print(generated_ids)

  preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]

  return "".join(preds), generated_ids

In [30]:
import torch

class BanglaRQADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = BanglaRQADataset(train_encodings)

In [ ]:
# Training part 
# no of epoch = 15
# opimizer = Adam
# at every epoch EM and F1 scores were calculated on validation set of BanglaRQA
# at every epoch model was also saved


from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

for epoch in range(15):
    # set model to train mode
    model.train()
    model_name = 'model_weights_epoch_' + str(epoch) + '.pth'
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        
        

        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['train_answer_encodings'].to(device)
        outputs = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                            labels = labels)
        
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update 
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': loss,
        
        }, os.path.join(checkpoint_dir,model_name))
    
    optim.zero_grad()
    model.eval()
    f1_total = 0.0
    em_total = 0.0

    with torch.no_grad():
      l = len(context_val)
      for i in range(l):
          question = {
              "context": context_val[i],
              "question": question_val[i]
              }

          pred, ids = get_answer(question["question"],question["context"])
          f1_total = f1_total + compute_f1(pred, answer_val[i])
          em_total = em_total + compute_exact_match(pred, answer_val[i])

      print("Epoch", epoch)
      print('F1: ', f1_total/l)
      print('EM: ', em_total/l)  

C:\Users\RAZER\anaconda3\envs\TitanoBoa\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/11976 [00:00<?, ?it/s]C:\Users\RAZER\AppData\Local\Temp\ipykernel_79260\4134177422.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 11976/11976 [1:37:09<00:00,  2.05it/s, loss=3.7]  


Epoch 0
F1:  0.24123989218328842
EM:  0.24123989218328842


Epoch 1: 100%|██████████| 11976/11976 [3:51:45<00:00,  1.16s/it, loss=2.18]       


Epoch 1
F1:  0.5548486146526651
EM:  0.42452830188679247


Epoch 2: 100%|██████████| 11976/11976 [1:36:38<00:00,  2.07it/s, loss=0.0702] 


...

Epoch 6: 100%|██████████| 11976/11976 [1:36:14<00:00,  2.07it/s, loss=0.0054]  


Epoch 6
F1:  0.7825225146593802
EM:  0.6064690026954178


Epoch 7: 100%|██████████| 11976/11976 [1:36:17<00:00,  2.07it/s, loss=0.000189]


Epoch 7
F1:  0.7685402481974268
EM:  0.5889487870619946


Epoch 8: 100%|██████████| 11976/11976 [6:02:49<00:00,  1.82s/it, loss=0.00173]     
